# Lesson - WPS
# Defining the model domain and initial conditions using the WRF Preprocessing System (WPS) and associated tools

## Overview
This lesson covers creating your case-specific domain and defining static and initial conditions used for the land surface model within WRF-Hydro. 

### Software and conventions
The lesson uses the [wrfhydro/wps:conus-training-v5.1.1](https://hub.docker.com/r/wrfhydro/wps) Docker container. There are both standard and training versions of the wrfhydro/wps container. The former run as a service using the directions posted on the [wrfhydro/wps DockerHub landing page](https://hub.docker.com/r/wrfhydro/wps) and the latter include the JupyterLab server allowing users to work with the container interactively and through training notebooks.

The containers include the following:
* Ubuntu base image
* WRF and WPS v3.9 built with the GNU Fortran compiler ‘gfortran’
* Python 3.6 command line utility for creating WRF-Hydro geogrid files using the WPS geogrid.exe program
* R environment including all libraries for running create_wrfinput and create_soilproperties scripts
* WRF-WPS geographical input data for the contiguous United States ONLY USGS+DEFAULT DATASETS ARE SUPPORTED, lai_modis_30s, nlcd2011_30m, and topo_30s are subsets for training purposes only

To complete this tutorial you may either execute commands by running each cell of this notebook. Alternatively, you may open a terminal in JupyterLab by selecting `File -> New -> Terminal` in JupyterLab and input the commands manually if you prefer. You can also use your own terminal by logging into the container with the command `docker exec -it wps-training bash`

All paths used in this lesson assume that the lesson materials are located under your home directory in a folder named `wrf-hydro-training`. If your materials are located in another directory, you will not be able to run the commands in this notebook inside Jupyter and will need to type them manually in your terminal session.

## Defining the model domain and creating the geogrid file
WRF-Hydro leverages the extensive domain generation and geophysical dataset processing tools available in the [WRF Preprocessing System (WPS)](http://www2.mmm.ucar.edu/wrf/users/) to define model domains and generate input files for the land surface model. In particular WRF-Hydro requires the geogrid and wrfinput files derived from these WPS utilities and in the case of wrfinput a utility within WRF as well. 

However, this requires that WRF-Hydro users build WRF and WPS on their systems, unpack the large datasets required for these utilties, and obtain meterological data required to run utilities intended to be used for the initialization of atmospheric models. Additionally, much of the functionality of WRF and WPS is not utilized for creating a geogrid file for WRF-Hydro, and many of the options in the *namelist.wps* are not relevant to this process. Therefore, we have created a Docker container, Python command line utility, and several script packages to abstract much of the WRF/WPS complexity and simplify the process of creating a these files for WRF-Hydro users.

This tutorial walks users through the process of manually creating a geogrid file from an existing WPS installation and creating wrfinput files from provided script packages as well as using the Python utility to generate geogrid and wrfinput files for a more automated approach.  For those interested in the details of running the complete WPS system and generating wrfinput files through WRF this is covered in our Docker training on the coupled WRF and WRF-Hydro modeling system.

### Data sources
The WPS geographical input data (e.g. land cover, soil texture, and terrain height) are some of the primary datasets used by WRF-Hydro. The entire dataset collection can be obtained from the WPS geographical input data [download page](http://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html). However, these data are rather large (~50GB) and contain a number of datasets that are not used in most implementations of WRF-Hydro. 

To facilitate the domain generation process, the WRF-Hydro team has created a subset of the data by removing unnecessary data, removing resolutions of the data not commonly used, and limiting the areal extent of this subset. This is consistent with the availability of other datasets used in this tutorial. Thus, the dataset provided with the wrfhydro/wps Docker container is considerably smaller than the complete WRF-WPS dataset.

### geogrid.exe
The WPS program *geogrid.exe* is used to create the *geo_em.d01.nc* file, hereafter referred to as the 'geogrid' file. The *geogrid.exe* program takes a Fortran namelist (*namelist.wps*) defining the projection, resolution, and extent of the model domain as well as the desired input fields and the [WPS geographical input data](http://www2.mmm.ucar.edu/wrf/users/download/get_sources_wps_geog.html) as inputs and creates the geogrid file. 

### Defining domain boundaries
The land surface model in WRF-Hydro operates on the domain defined within the geogrid file. The geogrid.exe program generates this file based upon input variables specified in the *namelist.wps* file. The first step to creating the geogrid file is to define our domain boundaries. The *geogrid.exe* program takes a centerpoint, x and y grid specifications, and other projection information to define a bounding box for the domain. If you do not already have coordinates, you can obtain them from Google Earth or a standard GIS system. However, for this tutorial we have supplied the coordinates for you and you may use the WRF-Hydro geogrid Python command line utility to visualize the domain location and create the geogrid file given these coordinates.

### Exploring the WPS geogrid.exe utility and namelist.wps
The WPS *geogrid.exe* utility is controlled by options set in the *namelist.wps*. As previously stated, there are many options in the *namelist.wps* file that are not relevant to most WRF-Hydro users. For educational purposes, we will take a quick look at a full *namelist.wps* file and then discuss the relevant options used by the WRF-Hydro geogrid utility in more detail.

Both the full namelist and the shorter annotated version intended for the WRF-Hydro geogrid utility are included within the example case for each WRF-Hydro release so we first need to set up our lesson directory and then download and unpack our the files to view these namelists.

#### Setting up the lesson directory

**Step 1: Setup the directory for this lesson**

First, we need to create a directory to hold the inputs and outputs for this lesson.

In [ ]:
%%bash
mkdir -p $HOME/wrf-hydro-training/lesson-wps

**Step 2: Download the example case.** 
    
The example case can be obtained from the [WRF-Hydro release page](https://github.com/NCAR/wrf_hydro_nwm_public/releases).

In [ ]:
%%bash
cd $HOME/wrf-hydro-training

wget https://github.com/NCAR/wrf_hydro_nwm_public/releases/download/v5.1.1-beta/croton_NY_example_testcase.tar.gz &> /dev/null
tar -xf croton_NY_example_testcase.tar.gz

In [ ]:
%%bash

cat $HOME/wrf-hydro-training/example_case/supplemental/namelist.wps.FULL_VERSION

A full description of these options can be found in the [WPS User Guide](http://www2.mmm.ucar.edu/wrf/users/docs/user_guide_V3.9/users_guide_chap3.html). 

The options that are of interest to most WRF-Hydro users are the following:

**Table 1. WPS namelist options relevent to WRF-Hydro**

|Option|Description|
|------|-----------|
| e_we | The number of grid edges in the west-east dimension. The number of grid centers will be e_we-1. |
| e_sn | The number of grid edges in the south-north dimension. The number of grid centers will be e_sn-1. |
| ref_lat | A real value specifying the latitude part of a (latitude, longitude) center-point of the domain. |
| ref_lon | A real value specifying the longitude part of a (latitude, longitude) center-point of the domain. West longitudes are negative, and the value of ref_lon should be in the range [-180, 180]. |
| dx | A real value specifying the grid distance in the x-direction where the map scale factor is 1. The grid distance is in meters for the 'polar', 'lambert', and 'mercator' projection, and in degrees longitude for the 'lat-lon' projection. |
| dy | A real value specifying the grid distance in the y-direction where the map scale factor is 1. The grid distance is in meters for the 'polar', 'lambert', and 'mercator' projection, and in degrees latitude for the 'lat-lon' projection. |
| map_proj | A character string specifying the projection of the simulation domain. Accepted projections are 'lambert', 'polar', 'mercator', and 'lat-lon'. Default value is 'lambert'. |
| truelat1 | A real value specifying, the first true latitude for the Lambert conformal conic projection, or the only true latitude for the Mercator and polar stereographic projections. |
| truelat2 | A real value specifying, the second true latitude for the Lambert conformal conic projection. For all other projections, truelat2 is ignored. No default value. |
| stand_lon | A real value specifying, the longitude that is parallel with the y-axis in the Lambert conformal and polar stereographic projections. For the regular latitude-longitude projection, this value gives the rotation about the earth's geographic poles. No default value. |
| geog_data_res | A character string specifying a corresponding resolution or list of resolutions separated by + symbols of source data to be used when interpolating static terrestrial data to the grid. This string should contain a resolution matching a string preceding a colon in a rel_path or abs_path specification (see the description of GEOGRID.TBL options) in the GEOGRID.TBL file for each field. If a resolution in the string does not match any such string in a rel_path or abs_path specification for a field in GEOGRID.TBL, a default resolution of data for that field, if one is specified, will be used. If multiple resolutions match, the first resolution to match a string in a rel_path or abs_path specification in the GEOGRID.TBL file will be used. Default value is 'default'. |
| geog_data_path | A character string giving the path, either relative or absolute, to the directory where the geographical data directories may be found. This path is the one to which rel_path specifications in the GEOGRID.TBL file are given in relation to. No default value. |


To simplify the process of creating the geogrid file, these options are the only ones needed when using the wrfhydro/wps Docker utility. 

Lets take a look at the namelist.wps file used by the `wrfhydro/wps` docker utility

In [ ]:
%%bash

cat $HOME/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED

#### Changing the domain location and boundaries in WPS
In this section we will make a few changes to the example case `namelist.wps` to demonstrate how to move the domain location and change the extent. 

**NOTE: Do not edit the *~/wrf-hydro-training/example_case/supplemental/namelist.wps* file because we will use this prepared namelist to create the example case geogrid file using WPS in a later section of this lesson**

In general, users want to generate the largest, highest-resolution model domain possible. Unfortunately, we are often constrained by available computational power or drive storage. These contraints usually define the acceptable grid sizes for us (*e_we*, *e_sn*). Therefore, once we know our domain center, we can play with *dx* and *dy* to cover our interested area.

We will edit the *dx* and *dy* options in the namelist to grow the size of the domain. We will leave all other options the same, which will simply increase the domain coverage but maintain the domain center and domain dimensions.

**Step 1: Visualize the original domain location.**

First let's create a basic map of the original example case domain location using the example case namelist.wps file *~/wrf-hydro-training/example_case/supplemental/namelist.wps* and the WRF-Hydro geogrid Python utility. You can get help on the arguments to the python utility by accessing its help.

In [ ]:
%%bash
python /home/docker/WRF_WPS/utilities/make_geogrid.py --help

Note that the script takes 4 arguments, the first `--namelist_path` is the path to your *namelist.wps* file, the second `--output_dir` is the path to the directory to hold the output, and the third `--plot_only` is a boolean `True` or `False` indicating whether or not to plot the domain. If `plot_only` the utility will only plot the domain and not create the geogrid file. This option is useful for making changes to your domain location or extent. The fourth argument `--create_wrfinput` is a boolean `True` or `False` indicating whether or not to create a wrfinput initial conditions.

Now, let's create a plot of the original example case domain location.

In [ ]:
%%bash
python /home/docker/WRF_WPS/utilities/make_geogrid.py \
--namelist_path /home/docker/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED \
--output_dir /home/docker/wrf-hydro-training/lesson-wps/ \
--plot_only

In [ ]:
from IPython.display import Image

Image('/home/docker/wrf-hydro-training/lesson-wps/domain.png')

**Step 2: Make a copy of the original domain namelist.wps**

We will copy our original domain namelist.wps that was used to create the example case and edit it. It is located at `~/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED` 

**NOTE: Do not edit the original namelist.wps `~/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED` because we will use it later to create our geogrid file.** 

In [ ]:
%%bash
cp ~/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED \
~/wrf-hydro-training/lesson-wps/namelist_edit.wps


**Step 3: Edit the `~/wrf-hydro-training/lesson-wps/namelist_edit.wps`**

Using your editor of choice (can be edited in Jupyter), open the `~/wrf-hydro-training/lesson-wps/namelist_edit.wps` file and change the *dx* and *dy* values to 
```
dx=2000.0
dy=2000.0
```

In [ ]:
%%bash
python /home/docker/WRF_WPS/utilities/make_geogrid.py \
--namelist_path /home/docker/wrf-hydro-training/lesson-wps/namelist_edit.wps \
--output_dir /home/docker/wrf-hydro-training/lesson-wps/ \
--plot_only

In [ ]:
from IPython.display import Image

Image('/home/docker/wrf-hydro-training/lesson-wps/domain.png')

We can see that the domain coverage has expanded but the center and size are the same. You can change any of your domain set-up by editing the variables in `namelist.wps`.


### Making the geogrid file
In this section, we will create the geogrid file using the WRF-Hydro geogrid python utility. 

**Note: If you would like to use this utilitiy outside of this training, the utility and all required datasets for the Continental United States are available in the *wrfhydro/wps* Docker container at https://hub.docker.com/r/wrfhydro/wps/.**

**Step 1: Copy the prepared *namelist.wps* file for the example case to the lesson-wps output folder**

We will copy over the prepared namelist.wps file from the example case to create our geogrid file. We will also rename our edited namelist.wps from the previous domain location exercise, in case you want to retain for reference.

In [ ]:
%%bash
cp /home/docker/wrf-hydro-training/example_case/supplemental/namelist.wps.SHORT_ANNOTATED \
~/wrf-hydro-training/lesson-wps/namelist.wps

**Step 1: Check the domain boundaries**

We will do a quick visual inspection to make sure our supplied domain boundary information is correct. 

In [ ]:
%%bash
python /home/docker/WRF_WPS/utilities/make_geogrid.py \
--namelist_path /home/docker/wrf-hydro-training/lesson-wps/namelist_edit.wps \
--output_dir /home/docker/wrf-hydro-training/lesson-wps/ \
--plot_only

In [ ]:
from IPython.display import Image

Image('/home/docker/wrf-hydro-training/lesson-wps/domain.png')

**Step 2: Create the geogrid and wrfinput files.**

This time we will run the utility with `--plot_only` set to False. A plot of the domain will still be produced, but the geogrid.exe utility will also be executed to create the geogrid file.

The wrfinput_d01.nc is a very basic WRF-Hydro initialization file (wrfinput) from a WRF geogrid file and a set of user-specified conditions. The file contains fields of spatially uniform initial model states of soil moisture, soil temperature, soil liquid water content and skin temperature among a few other variables necessary for model cold-start initialization. This file can be used as a 'cold start' for long-term model spin-up or users can overwrite the fields in the file created. Sophisticated and WRF-savvy users can bypass this script and use the WRF utility *real.exe* to create a wrfinput file from model or reanalysis products. 

The R script used to create this file can be downloaded at https://ral.ucar.edu/projects/wrf_hydro/pre-processing-tools.

**NOTE: This R script will be replaced in the future with a Python command line utility to eliminate the R dependency.**

In [ ]:
%%bash
python /home/docker/WRF_WPS/utilities/make_geogrid.py \
--namelist_path /home/docker/wrf-hydro-training/lesson-wps/namelist_edit.wps \
--output_dir /home/docker/wrf-hydro-training/lesson-wps/ 

### Inspecting the geogrid file
Now that we have created our geogrid file, we will quickly display the file header using the *ncdump* command line utility and open the file using the Python library `xarray` to visualize its contents. 

In [ ]:
%%bash

ncdump -h ~/wrf-hydro-training/lesson-wps/geo_em.d01.nc

In [ ]:
%matplotlib inline
import xarray as xr

geogrid = xr.open_dataset('~/wrf-hydro-training/lesson-wps/geo_em.d01.nc')
geogrid.HGT_M.plot()

**View the *wrfinput_d01.nc* file**

In [ ]:
%%bash
ncdump -h ~/wrf-hydro-training/lesson-wps/wrfinput_d01.nc

## Generating wrfinput files for use in WRF-Hydro

As mentioned above, the WRF-Hydro group also provides a script package to generate wrfinput files.  This section walks through how to download and run this script package.

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/lesson-wps

wget http://ral.ucar.edu/sites/default/files/public/create_wrfinput.zip &> /dev/null
unzip create_wrfinput.zip

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/lesson-wps/create_wrfinput/README.txt

In [ ]:
%%bash
cd /home/docker/wrf-hydro-training/lesson-wps/create_wrfinput

Rscript create_wrfinput.R --geogrid='/home/docker/wrf-hydro-training/lesson-wps/geo_em.d01.nc'

This concludes the lesson on domain definition and initial conditions for WRF-Hydro. 

© UCAR 2018